In [4]:
import glob
import numpy as np
import pandas as pd
from scipy import stats
from scipy.sparse import triu
from collections import defaultdict
from hicmatrix import HiCMatrix as hm
from statsmodels.stats.multitest import fdrcorrection
from matplotlib import cm
from matplotlib.colors import to_hex
from sklearn.preprocessing import KBinsDiscretizer

In [5]:
def directionPreference(matrix, chrom: str, binSize: int):

    diffChange = defaultdict(list)
    for start, df in matrix.groupby('start'):
        diffChange['start'].append(start)
        try:
            _, p = stats.wilcoxon(df['score'], alternative='two-sided')
        except ValueError:
            p = np.nan

        diffChange['p'].append(p)
        nonZeroScores = df.loc[df['score'] != 0, 'score']
        if not nonZeroScores.empty:
            direction = 1 if nonZeroScores.median() > 0 else -1
        else:
            direction = np.nan
        diffChange['direction'].append(direction)
    diffChange = pd.DataFrame(diffChange)
    diffChange['chrom'] = chrom
    diffChange['end'] = diffChange['start'] + binSize

    return diffChange

In [6]:
def getColour(x, alpha, colourmap, p='p(adj)'):
    if x[p] <= alpha:
        if x['direction'] == 1:
            i = (x['quantScore'] * 0.5)  + 0.5
        else:
            i = (1 - x['quantScore']) * 0.5
        colour = to_hex(cm.get_cmap(colourmap, 40)(i))[1:]
    else:
        i = x['quantScore']
        colour = to_hex(cm.get_cmap('binary', 20)(i))[1:]
    colour = f'{int(colour[:2], 16)},{int(colour[2:4], 16)},{int(colour[4:], 16)}'
    return colour

In [9]:
alpha = 0.05
colourmap = 'bwr'
maxDistance = 1000000
nBins = 20
est = KBinsDiscretizer(n_bins=nBins, encode='ordinal', strategy='kmeans')

In [10]:
for cell in ['GM12878', 'H1hESC', 'IMR90']:
    # Awaiting H1 results
    if cell == 'H1hESC':
        continue
        
    matrices = glob.glob(f'../../../HiCimages/{cell}/alleleGRCh37/dat/HiCsubtract/chr*/20000/{cell}_a1-vs-{cell}_a2-LOESSdiff-medianFilter-SNPsplit.h5')

    allRegions = []
    allDirection = []
    for matrix in matrices:
        hic = hm.hiCMatrix(matrix)
        binSize = hic.getBinSize()
        chrom = hic.getChrNames()[0]

        nonzeroIdx = hic.matrix.nonzero()
        nonzeroValues = hic.matrix[nonzeroIdx].tolist()[0]
        mat = pd.DataFrame({'start': nonzeroIdx[0], 'start2': nonzeroIdx[1], 'value': nonzeroValues})
        mat['start'] = mat['start'].apply(lambda x: hic.getBinPos(x)[1])
        mat['start2'] = mat['start2'].apply(lambda x: hic.getBinPos(x)[1])
        mat.columns = ['start', 'start2', 'score']
        mat['seperation'] = (mat['start'] - mat['start2']).abs()
        mat['abs(score)'] = abs(mat['score'])
        mat = mat.loc[mat['seperation'] <= maxDistance]
        
        binDirection = directionPreference(mat, chrom, binSize)
        # FDR correction by chromosome
        validP = binDirection['p'].notna()
        binDirection['p(adj)'] = np.nan
        binDirection.loc[validP, 'p(adj)'] = fdrcorrection(binDirection.loc[validP, 'p'])[1]
        allDirection.append(binDirection)

        summed = mat.groupby('start')[['abs(score)']].sum().reset_index()
        summed['chrom'] = chrom
        summed['quantScore'] = est.fit_transform(summed['abs(score)'].to_numpy().reshape(-1,1)) / nBins
        allRegions.append(summed)

    allRegions = pd.concat(allRegions).set_index(['chrom', 'start']).sample(frac=1)

    allDirection = pd.concat(allDirection).set_index(['chrom', 'start'])

    allRegions = allRegions.merge(
        allDirection, left_index=True, right_index=True).reset_index()
    allRegions['end'] = allRegions['start'] + binSize

    allRegions['colour'] = allRegions.apply(getColour, args=(alpha, colourmap), axis=1)

    allRegions['name'] = '.'
    allRegions['strand'] = '.'
    allRegions['thickStart'] = allRegions['start']
    allRegions['thickEnd'] = allRegions['end']
    columns = ([
        'chrom', 'start', 'end', 'name', 'abs(score)', 'strand',
        'thickStart', 'thickEnd', 'colour'])
    allRegions[columns].to_csv(
        f'/media/stephen/MyPassport1/PhD/HiCimages/{cell}/alleleGRCh37/{cell}-newComputeScore-reversed.bed', 
        index=False, header=False, sep='\t')

In [7]:
binDirection

,start,p,direction,chrom,end,p(adj)
0,2680000,1.000000,-1,chrX,2700000,1.000000
1,2700000,0.252104,-1,chrX,2720000,0.938945
2,2720000,0.655639,-1,chrX,2740000,1.000000
3,2740000,0.444709,-1,chrX,2760000,0.984587
4,2760000,0.391267,1,chrX,2780000,0.970456
...,...,...,...,...,...,...
7266,154840000,0.828785,1,chrX,154860000,1.000000
7267,154860000,0.769531,-1,chrX,154880000,1.000000
7268,154880000,0.300781,-1,chrX,154900000,0.950578
7269,154900000,0.473334,1,chrX,154920000,0.984587
